# Global imports

In [60]:
import pandas as pd
import numpy as np
import re, os, ast
from pathlib import Path

In [72]:
def parse_experiment_details(log_file):
    """
    Parses the experiment file to extract relevant data.
    """
    agent_settings = re.compile(r"^Parameters: (.*)$", re.MULTILINE)
    training_settings = re.compile(r"^Using settings: (.*)$", re.MULTILINE)

    agent_settings = agent_settings.search(log_file).group(1)
    training_settings = training_settings.search(log_file).group(1)
    computation_device = re.compile(r"^Device: (.*)$", re.MULTILINE)
    environment_name = re.compile(r"^Environment: (.*)$", re.MULTILINE)

    def parse_settings(settings_str):
        key_value_pattern = re.compile(r"(\w+)=((?:\[.*?\]|[^,]+))")
        return {
            key: ast.literal_eval(value) if "[" in value or value.lower() in ["true", "false", "none"] else value
            for key, value in key_value_pattern.findall(settings_str)
        }


    agent_settings = parse_settings(agent_settings)
    training_settings = parse_settings(training_settings)

    
    experiment_details = {
        **agent_settings,
        **training_settings,
        "computation_device": computation_device.search(log_file).group(1),
        "environment_name": environment_name.search(log_file).group(1),
    }

    return experiment_details


In [73]:
experiment_dir = Path("experiments")

experiment_regex = re.compile(r"(\w+-v5)_(\d)_(\d+-\d+-\d+_\d+-\d+-\d+)")

experiments = [dir for dir in os.listdir(experiment_dir) if experiment_regex.match(dir)]

evaluations = [experiment_dir.joinpath(experiment, "evaluations.npz") for experiment in experiments]
evaluations = map(np.load, evaluations)

exp_details = [experiment_dir.joinpath(experiment, "logs.txt") for experiment in experiments]
exp_details = map(lambda x: x.read_text(), exp_details)
exp_details = map(parse_experiment_details, exp_details)

exp_df = pd.concat([pd.DataFrame(evaluations), pd.DataFrame(exp_details)], axis=1)
exp_df.set_index("experiment_name")


,ep_lengths,results,timesteps,num_actors,gamma,tau,lr_q,lr_pi,lr_alpha,alpha_div,...,aggregation_method,updates_per_step,seed,time_steps,start_steps,eval_every,exp_dir,save_model,computation_device,environment_name
experiment_name,,,,,,,,,,,,,,,,,,,,,
HalfCheetah-v5_7_25-05-06_12-30-06,"[[1000, 1000, 1000, 1000, 1000, 1000, 1000, 10...","[[-557.5725480490727, -522.0415354866747, -527...","[11000, 12000, 13000, 14000, 15000, 16000, 170...",10,0.99,0.005,0.0003,0.0003,0.0003,0.01,...,max_q,1,7,1000000,10000,1000,/home/thompsjame1/code/AIML440_code/my-algorit...,True,cuda:2,HalfCheetah-v5
HalfCheetah-v5_7_25-05-06_12-32-07,"[[1000, 1000, 1000, 1000, 1000, 1000, 1000, 10...","[[-623.7724418810941, -657.6299009317746, -624...","[20000, 30000, 40000, 50000, 60000, 70000, 800...",9,0.99,0.005,0.0003,0.0003,0.0003,0.01,...,max_q,1,7,1000000,10000,10000,/home/thompsjame1/code/AIML440_code/my-algorit...,True,cuda:1,HalfCheetah-v5
HalfCheetah-v5_4_25-05-06_12-35-02,"[[1000, 1000, 1000, 1000, 1000, 1000, 1000, 10...","[[-547.4207505041388, -561.6317663969822, -561...","[20000, 30000, 40000, 50000, 60000, 70000, 800...",9,0.99,0.005,0.0003,0.0003,0.0003,0.01,...,max_q,1,4,1000000,10000,10000,/home/thompsjame1/code/AIML440_code/my-algorit...,True,cuda:1,HalfCheetah-v5
HalfCheetah-v5_4_25-05-06_13-32-36,"[[1000, 1000, 1000, 1000, 1000, 1000, 1000, 10...","[[-595.9220510457357, -603.4889518721321, -602...","[20000, 30000, 40000, 50000, 60000, 70000, 800...",3,0.99,0.005,0.0003,0.0003,0.0003,0.01,...,max_q,1,4,1000000,10000,10000,/home/thompsjame1/code/AIML440_code/my-algorit...,True,cuda:2,HalfCheetah-v5
Ant-v5_4_25-05-07_10-22-20,"[[1000, 1000, 342, 1000, 1000, 1000, 132, 156,...","[[-2802.8970638184633, -2849.5926744075728, -1...","[20000, 30000, 40000, 50000, 60000, 70000, 800...",9,0.99,0.005,0.0003,0.0003,0.0003,0.01,...,max_q,1,4,1000000,10000,10000,/home/thompsjame1/code/AIML440_code/my-algorit...,True,cuda:2,Ant-v5
Walker2d-v5_4_25-05-09_10-40-06,"[[7, 7, 6, 7, 6, 7, 6, 6, 6, 6], [73, 71, 71, ...","[[-6.150267969898977, -7.397905539577727, -7.2...","[20000, 30000, 40000, 50000, 60000, 70000, 800...",3,0.99,0.005,0.0003,0.0003,0.0003,0.01,...,max_q,1,4,1000000,10000,10000,/home/thompsjame1/code/AIML440_code/my-algorit...,True,cuda:1,Walker2d-v5
Walker2d-v5_4_25-05-07_10-17-42,"[[43, 43, 43, 43, 43, 43, 43, 43, 43, 31], [76...","[[1.9893222392425955, 1.2541308468922083, 2.30...","[20000, 30000, 40000, 50000, 60000, 70000, 800...",9,0.99,0.005,0.0003,0.0003,0.0003,0.01,...,max_q,1,4,1000000,10000,10000,/home/thompsjame1/code/AIML440_code/my-algorit...,True,cuda:0,Walker2d-v5
InvertedDoublePendulum-v5_4_25-05-09_11-46-31,"[[6, 4, 6, 5, 5, 6, 4, 4, 4, 6], [4, 4, 3, 3, ...","[[44.80842051596397, 25.47443019185136, 44.615...","[10000, 20000, 30000, 40000, 50000, 60000, 700...",3,0.99,0.005,0.0003,0.0003,0.0003,0.01,...,max_q,1,4,1000000,10000,10000,/home/thompsjame1/code/AIML440_code/my-algorit...,True,cuda:2,InvertedDoublePendulum-v5
Ant-v5_4_25-05-12_14-56-30,"[[126, 32, 16, 24, 38, 59, 94, 68, 20, 110], [...","[[-131.82762741191445, -22.238839225297696, -2...","[10000, 20000, 30000, 40000, 50000, 60000, 700...",3,0.99,0.005,0.0003,0.0003,0.0003,0.01,...,max_q,1,4,1000000,10000,10000,/home/thompsjame1/code/AIML440_code/my-algorit...,True,cuda:2,Ant-v5
